<a href="https://colab.research.google.com/github/Madhav-Ganorkar/Healthcare-Billing-Cost-Prediction-System/blob/main/Healthcare_Billing_Cost_Prediction_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🎯 Healthcare Billing Cost Prediction


In [35]:
# # Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
# # Load and Read the dataset
df = pd.read_csv("healthcare_dataset.csv")
df.sample(5)

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
46359,jOhN gRay,36,Male,B+,Cancer,2022-06-02,Jeffery Rodriguez,"Perez, Decker and Perez",Blue Cross,19445.544679,272,Emergency,2022-06-15,Paracetamol,Inconclusive
30585,TimOtHy mUnoz,61,Male,O+,Cancer,2024-04-27,Samantha Matthews,Kline Ltd,Blue Cross,29981.162688,186,Elective,2024-04-28,Paracetamol,Normal
49022,roBert KIm,22,Female,B-,Obesity,2024-01-24,Teresa Baker,Gentry-Walker,UnitedHealthcare,17450.388114,262,Urgent,2024-01-27,Aspirin,Inconclusive
26898,NAtaSha PEARsOn,71,Female,A-,Diabetes,2023-08-07,Kyle Lewis,"Rodriguez Glover and Brown,",Aetna,26625.497725,109,Urgent,2023-08-27,Ibuprofen,Inconclusive
39698,HolLY nGuyeN,56,Female,A-,Arthritis,2022-05-05,David Evans,and Sons Johnson,Blue Cross,28223.095280,273,Elective,2022-05-17,Penicillin,Abnormal


In [37]:
# Checking Data Shape
df.shape

(55500, 15)

In [38]:
# Standardize name formatting
df['Name'] = df['Name'].str.title()

In [39]:
# # Check for missing values
df.isnull().sum()

,0
Name,0
Age,0
Gender,0
Blood Type,0
Medical Condition,0
Date of Admission,0
Doctor,0
Hospital,0
Insurance Provider,0
Billing Amount,0


In [40]:
# Check for duplicate rows
dfi = df.duplicated()
print("No. of Duplicates:", dfi)

No. of Duplicates: 0        False
1        False
2        False
3        False
4        False
         ...  
55495    False
55496    False
55497    False
55498    False
55499    False
Length: 55500, dtype: bool


In [41]:
# Check basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55500 entries, 0 to 55499
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                55500 non-null  object 
 1   Age                 55500 non-null  int64  
 2   Gender              55500 non-null  object 
 3   Blood Type          55500 non-null  object 
 4   Medical Condition   55500 non-null  object 
 5   Date of Admission   55500 non-null  object 
 6   Doctor              55500 non-null  object 
 7   Hospital            55500 non-null  object 
 8   Insurance Provider  55500 non-null  object 
 9   Billing Amount      55500 non-null  float64
 10  Room Number         55500 non-null  int64  
 11  Admission Type      55500 non-null  object 
 12  Discharge Date      55500 non-null  object 
 13  Medication          55500 non-null  object 
 14  Test Results        55500 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 6.4

In [42]:
# Convert Date Columns
# Convert 'Date of Admission' and 'Discharge Date' to datetime format.
df["Date of Admission"] = pd.to_datetime(df['Date of Admission'])
df["Discharge Date"] = pd.to_datetime(df["Discharge Date"])

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55500 entries, 0 to 55499
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Name                55500 non-null  object        
 1   Age                 55500 non-null  int64         
 2   Gender              55500 non-null  object        
 3   Blood Type          55500 non-null  object        
 4   Medical Condition   55500 non-null  object        
 5   Date of Admission   55500 non-null  datetime64[ns]
 6   Doctor              55500 non-null  object        
 7   Hospital            55500 non-null  object        
 8   Insurance Provider  55500 non-null  object        
 9   Billing Amount      55500 non-null  float64       
 10  Room Number         55500 non-null  int64         
 11  Admission Type      55500 non-null  object        
 12  Discharge Date      55500 non-null  datetime64[ns]
 13  Medication          55500 non-null  object    

In [44]:
# Create New Features
df["Year_of_Admission"] = df["Date of Admission"].dt.year
df["Month_of_Admission"] = df["Date of Admission"].dt.month

In [45]:
df.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results,Year_of_Admission,Month_of_Admission
0,Bobby Jackson,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal,2024,1
1,Leslie Terry,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive,2019,8
2,Danny Smith,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal,2022,9
3,Andrew Watts,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal,2020,11
4,Adrienne Bell,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal,2022,9


In [48]:
df["Length_of_stay"] = (df["Discharge Date"] - df["Date of Admission"]).dt.days

In [49]:
# Binary feature example
df["Is_Emergency"] = df["Admission Type"].apply(lambda x: 1 if x == "Emergency" else 0)

In [50]:
df.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results,Year_of_Admission,Month_of_Admission,Length_of_stay,Is_Emergency
0,Bobby Jackson,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal,2024,1,2,0
1,Leslie Terry,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive,2019,8,6,1
2,Danny Smith,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal,2022,9,15,1
3,Andrew Watts,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal,2020,11,30,0
4,Adrienne Bell,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal,2022,9,20,0


**📊 Exploratory Data Analysis (Basic Visuals)**